In [1]:
import pandas as pd

path_in = '../resources/data_ignored/Verena/mEMA_merged_VJ_06052020.xlsx'
df = pd.read_excel(path_in)

# the results will be written to the following files
path_out_link_column = './../resources/data_ignored/Qualtrics_link.xlsx'

col_subject_id = 'subject_id_x'

print('unique', col_subject_id, len(df[col_subject_id].unique()))
print('total instances', len(df.index))
df = df.drop(df.index[df['objective_binge'] != True])
print('binge instances', len(df.index))

unique subject_id_x 42
total instances 1874
binge instances 186


In [2]:
print('unique binge', col_subject_id, len(df[col_subject_id].unique()))
objective_binge_subject_ids = df[col_subject_id].unique()
print(objective_binge_subject_ids)

# objective_binge_subjects.to_excel(path_out_link_column)

unique binge subject_id_x 38
[19312 19197 19180 19179 19176 19126 19039 19038 19036 19035 19034 18965
 18926 18924 18742 18736 18734 18727 18726 18699 18696 18689 18680 18639
 18579 18570 18445 18410 18402 18396 18392 18391 18246 18238 18228 18224
 18188 18124]


In [4]:
path_in_mapping = '../resources/data_ignored/Qualtrics_link/mapping_subject_id.xlsx'
path_in_questionaire = '../resources/data_ignored/Qualtrics_link/BtB_Study 1_NL_Merged_01052020_VJ.xlsx'
path_in_questionaire_out = '../resources/data_ignored/Qualtrics_link/BtB_Study 1_NL_Merged_01052020_VJ_binges.xlsx'

col_participant = 'participant' # please keep column names equal in mapping

df_mapping = pd.read_excel(path_in_mapping)
df_q = pd.read_excel(path_in_questionaire)

def get_mapped_participants():
    objective_binge_participants = []
    for subject_id in objective_binge_subject_ids:
        value = df_mapping.loc[df_mapping[col_subject_id] == subject_id, col_participant].iloc[0]
        objective_binge_participants.append(value)
        print(subject_id, value)
    return objective_binge_participants

objective_binge_participants = get_mapped_participants()

print(len(df_q.index))
print('unique', col_participant, len(df_q[col_participant].unique()))
df_q = df_q[~df_q[col_participant].isin(objective_binge_participants)]
print(len(df_q.index))
print('unique binge', col_participant, len(df_q[col_participant].unique()))

df_q.to_excel(path_in_questionaire_out)


19312 11c64b3de7113c44a4a1
19197 83782ac5d4f706c6a038
19180 6894fc27e082b4253280
19179 80cd75459a263096b294
19176 d825070268a54baf5c78
19126 53332af9f31c800b2ea8
19039 e4477f1981d4a1cb3bd4
19038 e422b74d885e74ea237e
19036 b1abb503f0faa20c1015
19035 63a78e2a2be39937b15b
19034 9a453fc69ca87a376ddd
18965 1bc17639f57ef1c19e2f
18926 89e97e5645a24e4f0430
18924 8c3d526bd116885bae8c
18742 7a83973b97a9eca3d0b7
18736 d61ea9093c90629b53f0
18734 31c5d5af795645c84806
18727 68ce6d809ca5fae05528
18726 c1558765d6a7dae29d78
18699 454cb10ab1f587b9d109
18696 2594eff948ddc2ea656a
18689 84b177ff3faeeaa1d6b0
18680 d4edd8b6e431e13102df
18639 e03fbbb0ac0df2f56dce
18579 a274733434a16d66bb95
18570 ab4a499261188dbbd09e
18445 d66456f61a2994728d19
18410 51457cb99ac22ba7c30d
18402 e8d1176374627ffdb96b
18396 6559f4bbfebbac54d03f
18392 1504ad074b95c8b7ed2d
18391 afc46f547dd64533deab
18246 4e080b7d9bbbfbd7cea5
18238 98aa0a841c81135588a8
18228 a6fcd60a6ab5e11d9d17
18224 4906c9bfe5351f17658c
18188 0bd5d6218902d1051ca2
1